In [24]:
import numpy as np
from scipy.optimize import minimize

# Constants
m = 2500  # mass of the probe (kg)
g = 1.625  # acceleration due to gravity on the Moon (m/s^2)
r = np.array([0.6, 0, 0])  # position vector of the thrusters (m)
F_E = np.array([0, 500, 0])  # force exerted by main thruster (N)
F_20 = np.array([[20, 0, 0], [0, 20, 0], [0, 0, 20], [0, 0, 20]])  # force exerted by each of the four 20-N thrusters (N)

# Inertia matrix and angular velocity
I_xx = m * (0.98 ** 2)
I_yy = m * (1.06 ** 2)
I_zz = m * (1.02 ** 2)
I = np.diag([I_xx, I_yy, I_zz])
w_desired = np.array([[0.04, 0, 0.06],
                      [0.06, 0, -0.04],
                      [0.06, 0, 0.02],
                      [0.06, 0, -0.02]])

# Function to minimize
def objective_function(F_times_t):
    return np.linalg.norm(np.cross(r, F_E) + np.sum(np.cross(r, F_20 * F_times_t[:, np.newaxis]), axis=0), axis=0)

# Constraints
def constraint_eq(F_times_t, w_d):
    r_shape = np.array(r).shape[0]  # Ensure r is a numpy array
    F_E_shape = np.array(F_E).shape[0]
    F_20_shape = np.array(F_20).shape[0]
    F_times_t_shape = np.array(F_times_t).shape[0]

    if r_shape != 3 or F_E_shape != 3 or F_20_shape != 4 or F_times_t_shape != 4:
        raise ValueError("Incompatible dimensions for cross product (dimension must be 3 for r, F_E, and 4 for F_20 and F_times_t)")

    return w_d - np.linalg.inv(I) @ (np.cross(r, F_E) + np.sum(np.cross(r, F_20 * F_times_t[:, np.newaxis]), axis=0))

# Solve for each desired angular velocity
for i, w_d in enumerate(w_desired, start=1):
    # Initial guess for optimization
    x0 = np.zeros(4)
    
    # Bounds for optimization (thruster activation times)
    bounds = [(0, None)] * 4
    
    # Constraints
    cons = {'type': 'eq', 'fun': lambda F_times_t: constraint_eq(F_times_t, w_d)}
    
    # Minimize the objective function
    result = minimize(objective_function, x0, method='SLSQP', bounds=bounds, constraints=cons)
    
    # Print results
    print(f"Scenario {i}:")
    print("Thruster Activation Times (seconds):", result.x)
    print("Minimum Objective Value:", result.fun)
    print()


Scenario 1:
Thruster Activation Times (seconds): [0. 0. 0. 0.]
Minimum Objective Value: 300.0

Scenario 2:
Thruster Activation Times (seconds): [0. 0. 0. 0.]
Minimum Objective Value: 300.0

Scenario 3:
Thruster Activation Times (seconds): [0. 0. 0. 0.]
Minimum Objective Value: 300.0

Scenario 4:
Thruster Activation Times (seconds): [0. 0. 0. 0.]
Minimum Objective Value: 300.0

